# Time Series Analysis and Forecasting

This is a series of notebooks to support lectures on _Time series analysis and forecast_.

![](https://user-images.githubusercontent.com/49638680/154160268-cf39a1ec-3557-4940-8853-d06fc7a79acf.png)

## Time series Forecast and Classification

In paticular, over this notebook we are going to use a Recurrent Neural Network architecture with _Attention Mechanism_ in order to implement a classifier of values associated to several time series, that will play the role of features.

### Some references

1. [Bahdanau Attention Mechanism](https://arxiv.org/abs/1409.0473)
2. [Transformer Architecture](https://arxiv.org/abs/1706.03762)
3. [Adversarial Sparse Transformer for Time Series Forecasting](https://proceedings.neurips.cc/paper/2020/file/c6b8c8d762da15fa8dbbdfb6baf9e260-Paper.pdf)
4. [Deep Transformer Models for Time Series Forecasting: The Influenza Prevalence Case](https://arxiv.org/abs/2001.08317)
5. [The Time Series Transformer](https://towardsdatascience.com/the-time-series-transformer-2a521a0efad3)
6. [Transformers for Time-series Forecasting](https://medium.com/mlearning-ai/transformer-implementation-for-time-series-forecasting-a9db2db5c820)

# Bahdanau Attention Mechanism

Consider a simple encoder-decoder newtork. This model is based on the one described in the [2014 `Seq2Seq` paper by Ilya Sutskever, Oriol Vinyals and Quoc V. Le](https://research.google/pubs/pub43155/).

![Image by the author](https://user-images.githubusercontent.com/49638680/156374648-1759a98c-34d2-418a-8b71-0254ca1c87ea.png)

The idea is to convert an _input sequence_ - in our case a time series, but it can be a sentence to be translated (as in the paper) - into an hidden latent vector and then use this as input for the decoder part and get the _output sequence_. Again, the output sequence can be a time series forecast or the translated sentence into another language, for instance.
For the sake of clarity and to avoid confusion, in the following we are going to refer only to _sequences_ having in mind they can be time series, texts, audio tracks, etc.

To schematise:

* __Encoder__: Such part of the network takes the input sequence and convert it into a fixed-length (_i.e._ the latent space dimension) encoding vector.

* __Decoder__: Such part of the network takes the encoding vector and convert it into a variable-length output sequence.

In principle there is no prescription about the kind of cells Encoder and Decoder network should be composed of, however, usually these are LSTM or GRU to avoid vanishing/exploding gradients.

## Introduction

## The Attention Layer

# Transformers: Queries, Keys, Values

## Scaled dot-product Attention



For these kinds of tasks, a pretty straightforward procedure would be to use an autoregressive model of some kind (like *ARIMA*, *ARMAX*); these models allow us to take into account autocorrelations in a time series, and also can accept the deterministic features in the future (typically called “exogenous variables”). One limitation of ARMAX is that it is a linear model, and also one needs to specify the order of autocorrelations to be taken into account parametrically. 
LSTMs, instead, can learn nonlinear patterns, and are able to take into account autocorrelations in a nonparametric way!

Furthermore, Attention model allows us to take into account longer sequences avoind the information bottleneck problem and focusing on the most relevant part of the past in order to forecast the future behaviour of the time series.

In [1]:
# Load libraries
import tensorflow as tf

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = "retina"
sns.set()

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


# Importing data and building dataset

Let’s start with a practical example of a time series and look at the [FordA dataset from the UCR/UEA archive.](https://www.cs.ucr.edu/~eamonn/time_series_data_2018/). 

The dataset we are using here is called FordA. The data comes from the UCR archive. The dataset contains $3601$ training instances and another $1320$ testing instances. 
Each timeseries corresponds to a measurement of engine noise captured by a motor sensor. 

An example task, woud be the automatic detection of the presence of a specific issue with the engine. The problem is a balanced binary classification task. The full description of this dataset can be found [here](http://www.j-wichard.de/publications/FordPaper.pdf).

In [3]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

# In the dataset engines with issues are labeled with 1
# the ones without issues with -1, we transform these into 0. 
y_train[y_train == -1] = 0 
y_test[y_test == -1] = 0

#### Goal
> detect whether an engine has an issue, analysing its noise time series. 

For the sake of simplicity, let’s skip a lot of data cleaning/feature engineering steps one could apply to this dataset. 

#### Exercise

Make a simple analysis on this dataset. Try to decompose the time series and see whether there is a recurrent behaviour in anomalous engines.

## Build the model

We are ready to build the model based on the attention mechanism.

Our model processes a tensor of shape `(batch size, sequence_length, features)`, where `sequence_length` is the number of time steps and features is each input time series.

We are going to build an Attention Layer, _i.e._ a tensorflow layer object. This is done for the sake of compatibility.
Indeed, in the models we built previously, you can replace your classification RNN layers with this one: the inputs are fully compatible!

In [4]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.2):
    # Normalization and Attention
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = tf.keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = tf.keras.layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(res)
    x = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

The main part of our model is now complete. 

We can stack multiple of those `transformer_encode`r blocks and we can also proceed to add the final _Multi-Layer Perceptron_ classification head. 
Apart from a stack of Dense layers, we need to reduce the output tensor of the `TransformerEncoder` part of our model down to a vector of features for each data point in the current batch. A common way to achieve this is to use a pooling layer. For this example, a `GlobalAveragePooling1D` layer is sufficient.

**Note**: here we use only an attention model encoder, a fully transformer architecture would require also a decoder part. We leave the implementation of this as an exercise.
If you feel lost, try to have a look at [this](https://www.tensorflow.org/text/tutorials/transformer) Tensorflow tutorial.

In [8]:
def build_model(input_shape,
                head_size, 
                num_heads, 
                ff_dim, 
                num_transformer_blocks, 
                mlp_units, 
                dropout=0, 
                mlp_dropout=0.2):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = tf.keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    
    for dim in mlp_units:
        x = tf.keras.layers.Dense(dim, activation="relu")(x)
        x = tf.keras.layers.Dropout(mlp_dropout)(x)
    outputs = tf.keras.layers.Dense(n_classes, activation="softmax")(x)
    
    return tf.keras.Model(inputs, outputs, name="TimeSeriesTransformer")

### Train and Evaluate

We can now go on the juicy part: training.

First, we build and compile the model to have a check on dimensions.

In [9]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)

model.summary()

Model: "TimeSeriesTransformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 500, 1)]     0                                            
__________________________________________________________________________________________________
layer_normalization_16 (LayerNo (None, 500, 1)       2           input_3[0][0]                    
__________________________________________________________________________________________________
multi_head_attention_8 (MultiHe (None, 500, 1)       7169        layer_normalization_16[0][0]     
                                                                 layer_normalization_16[0][0]     
__________________________________________________________________________________________________
dropout_18 (Dropout)            (None, 500, 1)       0           multi_head_at

#### Some comments

As you can see, we have ~$93$k parameters to train. The input shape is given by the whole length of the input time series. We can imagine this to be the length of the time window in the case of a series forecast.

Let's move on and train such a model.

We want to give an earlystopping callback, in fact it is expected that this model can easily overfit otherwise.

In [11]:
es_callback = tf.keras.callbacks.EarlyStopping(patience=10, 
                                            monitor='val_sparse_categorical_accuracy', 
                                            mode='max', 
                                            verbose=1,  
                                            restore_best_weights=True)

In [ ]:
callbacks = [es_callback]

history = model.fit(x_train,
                    y_train,
                    validation_split=0.2,
                    epochs=200,
                    batch_size=64,
                    callbacks=callbacks);

### Learning curves
Let's procede to analyse learning curves of our model.

In [ ]:
print(history.history.keys())

plt.figure(figsize=(15,10))

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.figure(figsize=(15,10))
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left');

### Model evaluation

In [11]:
model.evaluate(x_test, y_test, verbose=1)

2756/2756 [==============================] - 43s 16ms/step - loss: 0.0088 - mae: 0.0956


[0.008798547089099884, 0.09561186283826828]

## Some observations

In about $110$-$120$ epochs (~$25$s each on Colab with enabled GPU), the model reaches a training accuracy of ~$0.95$, validation accuracy of ~$0.84$ and a testing accuracy of ~$0.85$, without hyperparameter tuning. And that is for a model with less than $100$k parameters. Of course, parameter count and accuracy could be improved by a hyperparameter search and a more sophisticated learning rate schedule, or a different optimizer.

---

## Exercises 

1. Perform hyperparameter finetuning (you can use  a tool like the holy spirit, grid search or a framework like [optuna](https://optuna.org) if you like).

2. Implement the same model using an LSTM based architecture and compare results.

3. Apply the transformer architecture to a time series forecast problem

---

# Conclusion

The final model shows very good results on the data-set, this would outperform an implementation based on LSTM, which would suffer from short-term memory. Moreover, the architecture allows for much more rapid training, as the computations during training are done concurrently rather than sequentially. 

It can be concluded that the Transformer architecture, which is traditionally applied to NLP problems, has large potential in time series forecasting.